### Trabalho 2 - Control Flow Automaton
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado

Este programa implementa a multiplicação de dois inteiros $\,a,b\,$ , fornecidos como “input”,    e com precisão limitada a $\,n\,$ bits (fornecido como parâmetro do programa). Note-se que

 - Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”. 
 - Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um “output” com as operações indicadas
 - Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo


  1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste autómato. Para isso identifique as variáveis do modelo, o estado inicial e a relação de transição.
  2. Verifique se$\;$ $\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;$ $\;$é um invariante deste comportamento.

### Implementação

Começamos por importar o módulo `pysmt.shortcuts` que oferece uma API simplificada que disponibiliza as funcionalidades para a utilização usual de um SMT solver.
Os tipos estão definidos no módulo `pysmt.typing` de onde temos que importar o tipo `INT`. Definição dos inputs de $a$, $b$.

In [3]:
from pysmt.shortcuts import *
from pysmt.typing import INT

a,b = 5,10

A seguinte função cria a $i$-ésima cópia das variáveis de estado, agrupadas num dicionário que nos permite aceder às mesmas pelo nome.


In [2]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),INT)
    state['y'] = Symbol('y'+str(i),INT)
    state['z'] = Symbol('z'+str(i),INT)
    return state

Definição da função `init` que, dado um possível estado do programa (um dicionário de variáveis), devolva um predicado do pySMT que testa se esse estado é um possível estado inicial do programa.

In [4]:
def init(state):
    return And(Equals(state['pc'], Int(0)), Equals(state['x'], Int(a)), Equals(state['x'], Int(b)), Equals(state['x'], Int(0)))

Definição da função `trans` que, dados dois possíveis estados do programa, devolva um predicado do pySMT que testa se é possível transitar do primeiro para o segundo.

In [5]:
def trans(curr,prox):
    t0 = And(
        Equals(curr['pc'], Int(0)), 
        Equals(prox['pc'], Int(1)), 
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    t1 = And(
        Equals(curr['pc'], Int(1)), 
        NotEquals(curr['y'], Int(0)),
        Equals(curr['y'] % 2, Int(1)),
        Equals(prox['pc'], Int(2)), 
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y'] - 1),
        Equals(prox['z'], curr['x'] + curr['z'])
    )
    t2 = And(
        Equals(curr['pc'], Int(2)), 
        Equals(prox['pc'], Int(1)), 
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    t3 = And(
        Equals(curr['pc'], Int(1)), 
        NotEquals(curr['y'], Int(0)),
        Equals(curr['y'] % 2, Int(0)),
        Equals(prox['pc'], Int(3)), 
        Equals(prox['x'], 2 * curr['x']),
        Equals(prox['y'], curr['y'] / 2),
        Equals(prox['z'], curr['z'])
    )
    t4 = And(
        Equals(curr['pc'], Int(3)), 
        Equals(prox['pc'], Int(1)), 
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    # stop
    t5 = And(
        Equals(curr['pc'], Int(1)), 
        Equals(curr['y'], Int(0)),
        Equals(prox['pc'], Int(4)), # 4º estado
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    # no estado do stop
    t6 = And(
        Equals(curr['pc'], Int(4)), 
        Equals(prox['pc'], Int(4)), 
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
    
    # overflow - como faço?
    t7 = And(  
        Equals(curr['pc'], Int(2)), 
        Equals(prox['pc'], Int(2)), 
        Equals(prox['x'], curr['x'])
    )
    
    return Or(t0,t1,t2,t3)